## Import

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import pickle
import random as rnd
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import seaborn as sns
import sklearn
import statsmodels.api as sm
import xgboost as xgb
from matplotlib.font_manager import FontProperties
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,
                                           QuadraticDiscriminantAnalysis)
from sklearn.ensemble import (AdaBoostRegressor, BaggingClassifier,
                              BaggingRegressor, ExtraTreesRegressor,
                              GradientBoostingRegressor, RandomForestRegressor,
                              VotingRegressor)
from sklearn.feature_selection import (RFE, SelectFromModel, SelectKBest,
                                       VarianceThreshold, chi2)
from sklearn.inspection import permutation_importance
from sklearn.linear_model import (ElasticNet, HuberRegressor, Lasso,
                                  LinearRegression, RANSACRegressor, Ridge,
                                  SGDRegressor, TheilSenRegressor)
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             classification_report, confusion_matrix, log_loss,
                             precision_score, recall_score, roc_auc_score, mean_squared_error, r2_score)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     ShuffleSplit, StratifiedKFold,
                                     cross_val_score, cross_validate,
                                     train_test_split)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (Binarizer, LabelEncoder, MinMaxScaler,
                                   Normalizer, OneHotEncoder,
                                   PolynomialFeatures, StandardScaler,
                                   normalize)
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from time import time


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Reading data:

datapath : "../data"

In [3]:
traindf = pd.read_csv('../data/train.csv', index_col="ID")
traindf.sample(2)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8960,City Hotel,0,37,2015,September,38,16,0,2,2,0.0,0,BB,NZL,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient-Party,95.548755,0,0,Check-Out,2015-09-18
67128,City Hotel,1,163,2016,October,42,15,0,1,1,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,Non Refund,34.0,NaN,0,Transient,98.371402,0,0,Canceled,2016-09-27


### Drop columns not in test data and Clean missing data
* Drop columns: `is_canceled`, `reservation_status`, `reservation_status_date`, where they aren't in test data

In [4]:
testCol = set(pd.read_csv('../data/test.csv', index_col="ID").columns)
removeCols = [col for col in traindf.columns if col not in testCol]
removeCols.remove("adr")
print(removeCols)

['is_canceled', 'reservation_status', 'reservation_status_date']


In [5]:
traindf = traindf.drop(removeCols, axis=1)
traindf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
77062,Resort Hotel,350,2016,December,52,19,3,8,1,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,21.642196,0,1
85674,City Hotel,29,2017,February,8,24,2,2,3,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,144.077937,0,0


### Missing data, processing agent, company, country data
The below code shows column name of missing data

In [6]:
tmp = pd.to_numeric((traindf.isnull().sum() / traindf.shape[0]).map('{:,.2f}'.format))
tmp[tmp > 0]

country    0.01
agent      0.14
company    0.94
dtype: float64

Probrabaly one booking with `company` is more unlikely to chacel, also the one with specific `agent`, so turn them to boolean

For `Country`, take the nlargest(10)

In [7]:
def Clean_Missing(traindf):
    traindf["children"] = traindf["children"].fillna(0)

    traindf["company"] = (~traindf["company"].isna()).astype("int64")
    traindf["agent"] = (~traindf["agent"].isna()).astype("int64")
    return traindf

    
def Train_Top_Country(traindf):
    TopCountrySet = set(traindf["country"].value_counts()[:10].index)
    traindf["country"] = traindf["country"].apply(lambda x:x if x in TopCountrySet else "Other")
    traindf = pd.get_dummies(traindf, prefix="", prefix_sep="", columns=["country"])
    return traindf, TopCountrySet

traindf = Clean_Missing(traindf)
traindf, TopCountrySet = Train_Top_Country(traindf)

In [8]:
traindf.sample(5)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,BEL,BRA,DEU,ESP,FRA,GBR,IRL,ITA,NLD,Other,PRT
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
59011,Resort Hotel,3,2016,September,36,2,0,2,2,0.0,0,BB,Direct,Direct,1,0,1,A,C,0,No Deposit,1,0,0,Transient,150.958796,1,2,0,0,0,0,0,0,0,0,0,0,1
53424,Resort Hotel,0,2016,July,31,30,2,1,2,0.0,0,BB,Online TA,TA/TO,0,0,0,A,D,0,No Deposit,1,0,0,Transient,193.313087,0,0,0,0,0,0,0,0,0,0,0,1,0
11214,City Hotel,11,2015,September,39,26,0,1,2,0.0,0,BB,Online TA,TA/TO,0,1,0,C,C,0,No Deposit,1,0,0,Contract,216.431696,0,1,0,0,0,0,0,0,0,0,0,0,1
61634,City Hotel,53,2016,September,38,17,2,1,2,0.0,0,BB,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,1,0,0,Transient,92.896953,0,0,0,0,0,0,0,1,0,0,0,0,0
89647,City Hotel,113,2017,March,12,19,2,4,1,1.0,0,BB,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,1,0,0,Transient,57.849393,0,1,0,1,0,0,0,0,0,0,0,0,0


### Feature Transformation

`children`: `children` + `babies`  
`stays_in_nights`: `stays_in_week_nights` + `stays_in_weekend_nights`    
`hotel`: Convert hotel as `Resort Hotel` to 0, `City Hotel` to 1  

Create new feature `is_same_room`: Check if `reserved_room_type` is same as `assigned_room_type`(If so 1, else 0)

drop: `arrival_date_year` -- While training, most of them are 0  
      `arrival_date_week_number` -- Too trivial  
      `arrival_date_day_of_month` -- Too trivial  

One hot encoding: `meal`  
One hot encoding: `reserved_room_type` and `assigned_room_type`  
One hot encoding: `arrival_date_month`  
One hot encoding: `market_segment` `distribution_channel` `deposit_type` `customer_type`

In [9]:
def Feature_Transform(traindf):
    traindf = pd.get_dummies(traindf, prefix="", prefix_sep="", columns=["arrival_date_month"])
    traindf["children"] = traindf["children"] + traindf["babies"]
    traindf = traindf.drop(["babies"], axis=1)
    traindf["hotel"] = traindf["hotel"].map({"Resort Hotel": 0, "City Hotel": 1})

    traindf = traindf.drop(columns=["arrival_date_year", "arrival_date_week_number", "arrival_date_day_of_month"])

    traindf["is_same_room"] = (traindf["reserved_room_type"] == traindf["assigned_room_type"]).map({True: 1, False: 0})
    traindf = pd.get_dummies(traindf, columns=["reserved_room_type", "assigned_room_type"], prefix=["reserved", "assigned"])

    traindf = pd.get_dummies(traindf, columns=["meal", "market_segment", "distribution_channel", "deposit_type", "customer_type"])
    return traindf

traindf = Feature_Transform(traindf)

## Final Adjustment

In [13]:
traindf["adr_too_low"] = traindf["adr"] < 10
traindf["adr_too_low"] = traindf["adr_too_low"].astype("uint8")

In [14]:
traindf.sample(2)

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,BEL,BRA,DEU,ESP,FRA,GBR,IRL,ITA,NLD,Other,PRT,April,August,December,February,January,July,June,March,May,November,October,September,stays_in_nights,is_same_room,reserved_A,reserved_B,reserved_C,reserved_D,reserved_E,reserved_F,reserved_G,reserved_H,reserved_L,reserved_P,assigned_A,assigned_B,assigned_C,assigned_D,assigned_E,assigned_F,assigned_G,assigned_H,assigned_I,assigned_K,assigned_L,assigned_P,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,too_low,adr_too_low
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
89799,1,151,1,1,1,0.0,0,0,0,1,1,0,0,87.846458,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,False,0
44661,1,163,2,2,1,0.0,0,0,0,0,1,0,0,106.106282,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,4,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,False,0


### Train Model and Evaluate best model performance:

Since I only know about regression, I only use SVR as my model, and save the result as 

In [16]:
new_train_x = traindf.drop("adr", axis=1).values
new_train_y = traindf["adr"].values

In [49]:
def HenHenModelSelection():
    regressors = [LinearRegression(),
                   Lasso(),
                   Ridge()
#                    LinearSVR()
                   ]

    from time import time
    for regr in regressors:

        before = time()
        print("="*30)
        print(regr.__class__.__name__)

        X_train, X_test, y_train, y_test = train_test_split(new_train_x, new_train_y, test_size=0.2)
        scaler = StandardScaler()
        scaler = scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        regr.fit(X_train, y_train)
        X_test = scaler.transform(X_test)
        y_pred = regr.predict(X_test)
    
        after = time()

        print(f"MSE: {mean_squared_error(y_test, y_pred)} \nR2_score: {r2_score(y_test, y_pred)}")
        print(f"花費{(after - before):.1f}s")
        
#for i in range(10):
#    HenHenModelSelection()

In [17]:
def AaronModelSelection():
    classifiers = [LinearRegression(),
                   ElasticNet(),
                   Lasso(),
                   Ridge(),
                   LinearSVR(),
                   HuberRegressor(),
                   RANSACRegressor(),
    #                TheilSenRegressor(),
                   AdaBoostRegressor(),
    #                BaggingRegressor(),
                   GradientBoostingRegressor(),
                   RandomForestRegressor(),
    #                DecisionTreeRegressor(),
                   KNeighborsRegressor(),
                   XGBRegressor()
                   ]

    from time import time
    from sklearn.pipeline import make_pipeline

    for clf in classifiers:
        before = time()
        print("="*30)
        print(clf.__class__.__name__)
        
        pipe = make_pipeline(StandardScaler(), clf)
        crossValScore = cross_validate(pipe, new_train_x, new_train_y, scoring=('r2', 'neg_mean_squared_error'), cv=5) 
        for metric, value in crossValScore.items():
            print(f"{metric}: {abs(np.mean(value)):.2f}")
        after = time()
        print(f"花費{(after - before):.1f}s")

    print("="*30)
    
AaronModelSelection()

LinearRegression
fit_time: 0.70
score_time: 0.05
test_r2: 102139667209066857365504.00
test_neg_mean_squared_error: 256404705825670301275389952.00
花費3.8s
ElasticNet
fit_time: 0.64
score_time: 0.05
test_r2: 0.41
test_neg_mean_squared_error: 1501.12
花費3.5s
Lasso
fit_time: 0.70
score_time: 0.04
test_r2: 0.43
test_neg_mean_squared_error: 1459.68
花費3.8s
Ridge
fit_time: 0.53
score_time: 0.05
test_r2: 0.45
test_neg_mean_squared_error: 1387.34
花費3.0s
LinearSVR
fit_time: 12.21
score_time: 0.04
test_r2: 0.43
test_neg_mean_squared_error: 1454.05
花費61.4s
HuberRegressor
fit_time: 2.63
score_time: 0.04
test_r2: 0.44
test_neg_mean_squared_error: 1429.87
花費13.4s
RANSACRegressor
fit_time: 1.97
score_time: 0.05
test_r2: 16075199825887398572589056.00
test_neg_mean_squared_error: 35885737168905344071659159552.00
花費10.2s
AdaBoostRegressor
fit_time: 16.96
score_time: 0.31
test_r2: 0.01
test_neg_mean_squared_error: 2541.89
花費86.4s
GradientBoostingRegressor
fit_time: 31.55
score_time: 0.08
test_r2: 0.54
test_n

KeyboardInterrupt: 

In [51]:
regr = GradientBoostingRegressor()
scaler = StandardScaler()
scaler = scaler.fit(new_train_x)
new_train_x = scaler.transform(new_train_x)
regr.fit(new_train_x, new_train_y)

GradientBoostingRegressor()

In [52]:
testdf = pd.read_csv('../data/test.csv', index_col="ID")

In [53]:
testdf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94369,Resort Hotel,144,2017,April,16,16,1,0,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,242.0,NaN,0,Transient,0,0
107905,Resort Hotel,49,2017,June,25,24,1,1,3,0.0,0,HB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,15.0,NaN,0,Transient,0,0


In [54]:
testdf = Clean_Missing(testdf)
testdf = Feature_Transform(testdf)

testdf["country"] = testdf["country"].apply(lambda x:x if x in TopCountrySet else "Other")
testdf = pd.get_dummies(testdf, prefix="", prefix_sep="", columns=["country"])

In [55]:
testdf.sample(2)

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,April,August,July,June,May,stays_in_nights,is_same_room,reserved_A,reserved_B,reserved_C,reserved_D,reserved_E,reserved_F,reserved_G,reserved_H,reserved_P,assigned_A,assigned_B,assigned_C,assigned_D,assigned_E,assigned_F,assigned_G,assigned_H,assigned_I,assigned_K,assigned_P,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,BEL,BRA,DEU,ESP,FRA,GBR,IRL,ITA,NLD,Other,PRT
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
110508,1,120,2,3,3,0.0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,5,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
101198,1,420,2,1,2,0.0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [56]:
for col in set(traindf.columns) ^ set(testdf.columns):
    testdf[col] = np.zeros(len(testdf))

In [57]:
testdf_x = testdf.drop(["adr"], axis=1)
testdf_x = scaler.transform(testdf_x)

y_pred = regr.predict(testdf_x)
testdf["adr"] = y_pred

TestAdr = pd.DataFrame(y_pred, columns=["adr"]) 
TestAdr.to_csv("..//data//TestAdr.csv", index=False)

## End of ducument